In [1]:
import time
import random
import json
import datetime
import logging
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException,\
    TimeoutException, NoSuchElementException, StaleElementReferenceException

In [2]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.delete_all_cookies()
driver.get('https://www.jalan.net/biz/130000/')

In [ ]:
# link2-> https://www.jalan.net/biz/130000/page2.html

In [4]:
# Parse page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
main_section = soup.find('div', class_='styleguide-scope p-searchResults')
main_section

In [19]:
# Base URL for constructing full links
base_url = 'https://www.jalan.net/'

# List to hold extracted data
hotel_data = []

main_section = soup.find('div', class_='styleguide-scope p-searchResults')
if main_section:
    # Extract hotel items
    hotel_items = main_section.find_all('ol')
    for ol in hotel_items:
        li_elements = ol.find_all('li', class_ = 'p-yadoCassette')
    
        for item in li_elements:
            hotel_name_element = item.find('h2', class_='p-searchResultItem__facilityName')
            location_element = item.find('dd', class_='p-searchResultItem__accessValue')
            price_element = item.find('span', class_='p-searchResultItem__lowestPriceValue')
            plans_table = item.find('table', class_='p-planTable p-searchResultItem__planTable')
            
            hotel_name = hotel_name_element.get_text(strip=True) if hotel_name_element else 'N/A'
            hotel_link = base_url + hotel_name_element.find('a')['href'] if hotel_name_element else 'N/A'
            location = location_element.get_text(strip=True) if location_element else 'N/A'
            price = price_element.get_text(strip=True) if price_element else 'N/A'
            
            # Extract accommodation plans
            plans = []
            if plans_table:
                rows = plans_table.find_all('tr')[1:]  # Skip header row
                for row in rows:
                    plan_name_element = row.find('a', class_='p-searchResultItem__planName')
                    per_person_element = row.find('span', class_='p-searchResultItem__perPerson')
                    total_element = row.find('span', class_='p-searchResultItem__total')
                    
                    plan_name = plan_name_element.get_text(strip=True) if plan_name_element else 'N/A'
                    plan_link = base_url + plan_name_element['href'] if plan_name_element else 'N/A'
                    per_person_price = per_person_element.get_text(strip=True) if per_person_element else 'N/A'
                    total_price = total_element.get_text(strip=True) if total_element else 'N/A'
                    
                    plans.append({
                        'Plan Name': plan_name,
                        'Plan Link': plan_link,
                        'Per Person Price': per_person_price,
                        'Total Price': total_price
                    })
            
            # Append data for each hotel item
            hotel_data.append({
                'Hotel Name': hotel_name,
                'Hotel Link': hotel_link,
                'Location': location,
                'Price': price,
                'Plans': plans
            })

# Print the extracted data
for hotel in hotel_data:
    print(f"Hotel Name: {hotel['Hotel Name']}")
    print(f"Hotel Link: {hotel['Hotel Link']}")
    print(f"Location: {hotel['Location']}")
    print(f"Price: {hotel['Price']}")
    print("Plans:")
    for plan in hotel['Plans']:
        print(f"  Plan Name: {plan['Plan Name']}")
        print(f"  Plan Link: {plan['Plan Link']}")
        print(f"  Per Person Price: {plan['Per Person Price']}")
        print(f"  Total Price: {plan['Total Price']}")
    print('---')

Hotel Name: ホテルサンターガス上野店
Hotel Link: https://www.jalan.net//yad339296/
Location: JR上野駅広小路口より徒歩４分、東京メトロ上野駅３番出口より徒歩３分
Price: 1円～
Plans:
  Plan Name: ☆70歳以上限定☆宿泊日当日誕生日プラン☆要写真付身分証☆
  Plan Link: https://www.jalan.net//uw/uwp3200/uww3201init.do?yadNo=339296&planCd=03339495&roomTypeCd=0200621&adultNum=2&dateUndecided=1
  Per Person Price: 1円～
  Total Price: 1円～
  Plan Name: ☆70歳以上限定☆宿泊日当日誕生日プラン☆要写真付身分証☆
  Plan Link: https://www.jalan.net//uw/uwp3200/uww3201init.do?yadNo=339296&planCd=03339495&roomTypeCd=0200608&adultNum=2&dateUndecided=1
  Per Person Price: 1円～
  Total Price: 1円～
  Plan Name: ☆３泊ごとのＥＣＯ清掃 シンプルステイプラン☆　※２泊からのご予約が可能です
  Plan Link: https://www.jalan.net//uw/uwp3200/uww3201init.do?yadNo=339296&planCd=03572700&roomTypeCd=0200621&adultNum=2&dateUndecided=1
  Per Person Price: N/A
  Total Price: N/A
---
Hotel Name: Ｊホテル東京ジオ
Hotel Link: https://www.jalan.net//yad357898/
Location: 東京駅、神田駅、大手町、三越前駅、徒歩5分～7分と各種最寄♪ディズニー最寄り駅の舞浜まで快速3駅14分可
Price: 1円～
Plans:
  Plan Name: 【数量限定】【素泊り】ウィークリープラン長期滞

In [1]:
import pandas as pd
df = pd.read_csv(r"./scraped_data/hotel_data_20240822_225836.csv")
df.head()

,Hotel Name,Hotel Link,Location,Price,Plan Name,Plan Link,Per Person Price,Total Price
0,ホテルサンターガス上野店,https://www.jalan.net/yad339296/,JR上野駅広小路口より徒歩４分、東京メトロ上野駅３番出口より徒歩３分,1円～,☆70歳以上限定☆宿泊日当日誕生日プラン☆要写真付身分証☆,https://www.jalan.net/uw/uwp3200/uww3201init.d...,1円～,1円～
1,ホテルサンターガス上野店,https://www.jalan.net/yad339296/,JR上野駅広小路口より徒歩４分、東京メトロ上野駅３番出口より徒歩３分,1円～,☆70歳以上限定☆宿泊日当日誕生日プラン☆要写真付身分証☆,https://www.jalan.net/uw/uwp3200/uww3201init.d...,1円～,1円～
2,ホテルサンターガス上野店,https://www.jalan.net/yad339296/,JR上野駅広小路口より徒歩４分、東京メトロ上野駅３番出口より徒歩３分,1円～,☆３泊ごとのＥＣＯ清掃 シンプルステイプラン☆ ※２泊からのご予約が可能です,https://www.jalan.net/uw/uwp3200/uww3201init.d...,NaN,NaN
3,Ｊホテル東京ジオ,https://www.jalan.net/yad357898/,東京駅、神田駅、大手町、三越前駅、徒歩5分～7分と各種最寄♪ディズニー最寄り駅の舞浜まで快速...,1円～,【数量限定】【素泊り】ウィークリープラン長期滞在でお得に♪ Wi-Fi環境有りです！,https://www.jalan.net/uw/uwp3200/uww3201init.d...,NaN,NaN
4,Ｊホテル東京ジオ,https://www.jalan.net/yad357898/,東京駅、神田駅、大手町、三越前駅、徒歩5分～7分と各種最寄♪ディズニー最寄り駅の舞浜まで快速...,1円～,【一人旅応援】【素泊り】クイーンベッドを独り占め♪東京メトロ24時間乗り放題乗車券付き！,https://www.jalan.net/uw/uwp3200/uww3201init.d...,"11,880円～","11,880円～"


In [2]:
df

,Hotel Name,Hotel Link,Location,Price,Plan Name,Plan Link,Per Person Price,Total Price
0,ホテルサンターガス上野店,https://www.jalan.net/yad339296/,JR上野駅広小路口より徒歩４分、東京メトロ上野駅３番出口より徒歩３分,1円～,☆70歳以上限定☆宿泊日当日誕生日プラン☆要写真付身分証☆,https://www.jalan.net/uw/uwp3200/uww3201init.d...,1円～,1円～
1,ホテルサンターガス上野店,https://www.jalan.net/yad339296/,JR上野駅広小路口より徒歩４分、東京メトロ上野駅３番出口より徒歩３分,1円～,☆70歳以上限定☆宿泊日当日誕生日プラン☆要写真付身分証☆,https://www.jalan.net/uw/uwp3200/uww3201init.d...,1円～,1円～
2,ホテルサンターガス上野店,https://www.jalan.net/yad339296/,JR上野駅広小路口より徒歩４分、東京メトロ上野駅３番出口より徒歩３分,1円～,☆３泊ごとのＥＣＯ清掃 シンプルステイプラン☆ ※２泊からのご予約が可能です,https://www.jalan.net/uw/uwp3200/uww3201init.d...,NaN,NaN
3,Ｊホテル東京ジオ,https://www.jalan.net/yad357898/,東京駅、神田駅、大手町、三越前駅、徒歩5分～7分と各種最寄♪ディズニー最寄り駅の舞浜まで快速...,1円～,【数量限定】【素泊り】ウィークリープラン長期滞在でお得に♪ Wi-Fi環境有りです！,https://www.jalan.net/uw/uwp3200/uww3201init.d...,NaN,NaN
4,Ｊホテル東京ジオ,https://www.jalan.net/yad357898/,東京駅、神田駅、大手町、三越前駅、徒歩5分～7分と各種最寄♪ディズニー最寄り駅の舞浜まで快速...,1円～,【一人旅応援】【素泊り】クイーンベッドを独り占め♪東京メトロ24時間乗り放題乗車券付き！,https://www.jalan.net/uw/uwp3200/uww3201init.d...,"11,880円～","11,880円～"
...,...,...,...,...,...,...,...,...
80,ホテルヒルトップ,https://www.jalan.net/yad335106/,東武東上線成増駅徒歩１分！,"4,400円～",男性限定＊カプセル＆サウナセットプラン(下段）,https://www.jalan.net/uw/uwp3200/uww3201init.d...,"4,500円～","4,500円～"
81,ホテルヒルトップ,https://www.jalan.net/yad335106/,東武東上線成増駅徒歩１分！,"4,400円～",男性限定！得々カプセル＆サウナプラン,https://www.jalan.net/uw/uwp3200/uww3201init.d...,-円,"4,500円～"
82,豪華カプセルホテル 安心お宿プレミア新宿駅前店,https://www.jalan.net/yad319308/,新宿駅東南口より徒歩90秒！/バスタ新宿より徒歩90秒！/新宿3丁目駅徒歩5分,"4,480円～",【男性専用】【7連泊以上でお得！】,https://www.jalan.net/uw/uwp3200/uww3201init.d...,NaN,NaN
83,豪華カプセルホテル 安心お宿プレミア新宿駅前店,https://www.jalan.net/yad319308/,新宿駅東南口より徒歩90秒！/バスタ新宿より徒歩90秒！/新宿3丁目駅徒歩5分,"4,480円～",【男性専用】【7連泊以上でお得！】,https://www.jalan.net/uw/uwp3200/uww3201init.d...,NaN,NaN
